In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 700 kB 10.7 MB/s 
     |████████████████████████████████| 596 kB 47.1 MB/s 
     |████████████████████████████████| 419 kB 49.6 MB/s 
     |████████████████████████████████| 5.8 MB 49.2 MB/s 
     |████████████████████████████████| 141 kB 53.2 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.9.1 which is inc

In [3]:
import pytorch_lightning as pl
import pandas as pd
import cv2
import os 
from torch import nn
from torch.utils.data import Dataset ,DataLoader, random_split
import numpy as np
import torch
from sklearn.model_selection import train_test_split 
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import torchmetrics
from torchmetrics.functional import accuracy
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger
from torchvision.utils import make_grid
import math
import torch.nn.functional as F
from torchsummary import summary

In [4]:
pl.seed_everything(7)
PATH_DATASETS = os.environ.get("PATH_DATASETS", "/content/drive/MyDrive/Datasets")
BATCH_SIZE = 256 if torch.cuda.is_available() else 64
CLASS_SIZE = 10
VAL_SIZE = 10000

Global seed set to 7


In [6]:
class Data(pl.LightningDataModule):
  def __init__(self, data_dir: str=PATH_DATASETS, batch_size: int=BATCH_SIZE, class_size: int=CLASS_SIZE,
               val_size: int=VAL_SIZE):
    super().__init__()
    self.data_dir = data_dir
    self.batch_size = batch_size
    self.class_size = class_size
    self.val_size = val_size
    self.transform = {
        'train': transforms.Compose([
            transforms.Resize((224,224)),
            transforms.RandomHorizontalFlip(p=0.7),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ]),

        'test': transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
    }

    

  def prepare_data(self):
    datasets.CIFAR10(root=self.data_dir, train=True, download=True)
    datasets.CIFAR10(root=self.data_dir, train=False, download=True)

  def setup(self, stage=None):
    if stage == 'fit' or stage is None:
      cifar10_full = datasets.CIFAR10(root=self.data_dir, train=True, transform=self.transform['train'])
      self.train_size = len(cifar10_full) - self.val_size
      self.cifar_train, self.cifar_val = random_split(cifar10_full, [self.train_size, self.val_size])

    if stage == 'test' or stage is None:
      self.cifar_test = datasets.CIFAR10(root=self.data_dir, train=False, transform=self.transform['test'])

  def train_dataloader(self):
    return DataLoader(self.cifar_train, batch_size=32, shuffle=True)

  def val_dataloader(self):
    return DataLoader(self.cifar_val, batch_size=32, shuffle=False)

  def test_dataloader(seld):
    return DataLoader(self.cifar_test, batch_size=32, shuffle=False)

  def visualize(self):
    for images, _ in self.train_dataloader():
      print(f"images.shape: {images.shape}")
      plt.figure(figsize=(16,8))
      plt.axis("off")
      plt.imshow(make_grid(images, nrow=16).permute(1, 2, 0))
      break

In [7]:
dm = Data()
dm.prepare_data()

Files already downloaded and verified
Files already downloaded and verified


In [8]:
dm.setup()

In [13]:
class LitResnet18(pl.LightningModule):
  def __init__(self, in_channels, outputs=10):
    super().__init__()
    self.save_hyperparameters()

    class ResBlock(nn.Module):
      def __init__(self, in_channels, out_channels, downsample):
        super().__init__()
        if downsample:
          self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1)
          self.shortcut = nn.Sequential(
              nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2),
              nn.BatchNorm2d(out_channels)
          )
        else:
          self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
          self.shortcut = nn.Sequential()

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

      def forward(self, x):
        shortcut = self.shortcut(x)
        x = nn.ReLU()(self.bn1(self.conv1(x)))
        x = nn.ReLU()(self.bn2(self.conv2(x)))
        x = x + shortcut
        return nn.ReLU()(x)

    self.layer0 = nn.Sequential(
        nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU()
    )

    self.layer1 = nn.Sequential(
        ResBlock(64, 64, downsample=False),
        ResBlock(64, 64, downsample=False) 
    )

    self.layer2 = nn.Sequential(
        ResBlock(64, 128, downsample=True),
        ResBlock(128, 128, downsample=False)
    )

    self.layer3 = nn.Sequential(
        ResBlock(128, 256, downsample=True),
        ResBlock(256, 256, downsample=False)
    )

    self.layer4 = nn.Sequential(
        ResBlock(256, 512, downsample=True),
        ResBlock(512, 512, downsample=False)
    )

    self.gap = nn.AdaptiveAvgPool2d(1)
    self.fc = nn.Linear(1, outputs)
  
  def forward(self, x):
    x = self.layer0(x)
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.gap(x)
    x = torch.flatten(x)
    x = self.fc(x)
    
    return x


  def training_step(self, batch, batch_idx):
    x, y = batch
    logits = self.forward(x)
    loss = nn.functional.cross_entropy(logits, y)
    self.log('train loss', loss)
    return loss

  def evaluate(self, batch, stage=None):
    x, y = batch
    logits = self.forward(x)
    loss = nn.functional.cross_entropy(logits, y)
    preds = torch.argmax(logits, dim=1)
    acc = accuracy(preds, y)

    if stage:
      self.log(f'{stage}_loss', loss, prog_bar=True)
      self.log(f'{stage}_acc', acc, prog_bar=True)

  def validation_step(self, batch, batch_idx):
    self.evaluate(batch, stage="val")

  def test_step(self, batch, batch_idx):
    self.evaluate(batch, stage="test")

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=1e-4)

In [14]:
model = LitResnet18(3, outputs=10)
model.to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
summary(model, (3, 32, 32))

RuntimeError: ignored